In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import TimeSeriesSplit
from tqdm import tqdm
import numpy as np


In [2]:
df = pd.read_csv('final_direct_train.csv')
valid = pd.read_csv('final_direct_valid.csv')

In [3]:
best_feature_set = ['speed_gps',
 'lon',
 'lat',
 'direction',
 'alt',
 'accel_angular_nn',
 'accel_vertical_nn',
 'location_cluster',
 'accel_forward_nn',
 'accel_braking_nn',
 'day',
 'day_of_week',
 'day_of_week_cos',
 'hour_cos',
 'hour',
 'mdm_object_name',
 'minute',
 'second',
 'month',
 'mdm_model_id']

In [4]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=8, random_state=42)
df['location_cluster'] = kmeans.fit_predict(df[['lon','lat']])
valid['location_cluster'] = kmeans.predict(valid[['lon','lat']])

c:\Users\Lenovo\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [5]:
df['create_dt'] = pd.to_datetime(df['create_dt'])
valid['create_dt'] = pd.to_datetime(valid['create_dt'])

In [6]:
df = df.sort_values(by='create_dt')
valid = valid.sort_values(by='create_dt')

In [7]:
# train drop all the rows with a label = 4 
print(df.shape)
df = df[df['label'] != 4]
print(df.shape)

(516070, 26)
(503102, 26)


In [26]:
best_params = {
    'bootstrap': True,
    'class_weight': "balanced",
    'criterion': 'entropy',
    'max_depth': 10,
    'min_samples_leaf': 1,
    'min_samples_split': 5,
    'n_estimators': 500
}

# Initialize the Random Forest classifier
rf = RandomForestClassifier(**best_params, random_state=42)

tscv = TimeSeriesSplit(n_splits=5)

X = df[best_feature_set]
y = df['label']

In [28]:
rf.fit(X, y)


RandomForestClassifier(class_weight='balanced', criterion='entropy',
                       max_depth=10, min_samples_split=5, n_estimators=500,
                       random_state=42)

In [27]:
X

,speed_gps,lon,lat,direction,alt,accel_angular_nn,accel_vertical_nn,location_cluster,accel_forward_nn,accel_braking_nn,day,day_of_week,day_of_week_cos,hour_cos,hour,mdm_object_name,minute,second,month,mdm_model_id
278480,0.0,0.005753,0.004778,326.0,-50,0.0,0.0,2,0.0,0.0,9,2,-0.222521,-0.500000,8,1038,2,59,8,62
278481,0.0,0.005753,0.004778,326.0,-50,0.0,0.0,2,0.0,0.0,9,2,-0.222521,-0.500000,8,1038,2,59,8,62
278482,0.0,0.005753,0.004778,326.0,-50,0.0,0.0,2,0.0,0.0,9,2,-0.222521,-0.500000,8,1038,3,4,8,62
278483,0.0,0.005753,0.004778,326.0,-50,0.0,0.0,2,0.0,0.0,9,2,-0.222521,-0.500000,8,1038,3,4,8,62
278484,0.0,0.005753,0.004778,326.0,-50,0.0,0.0,2,0.0,0.0,9,2,-0.222521,-0.500000,8,1038,3,9,8,62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278073,0.0,0.007083,0.006013,270.0,-55,0.0,0.0,2,0.0,0.0,15,4,-0.900969,-0.707107,15,125,54,21,12,61
278074,0.0,0.007083,0.006013,270.0,-55,0.0,0.0,2,0.0,0.0,15,4,-0.900969,-0.707107,15,125,54,26,12,61
278075,0.0,0.007083,0.006013,270.0,-55,0.0,0.0,2,0.0,0.0,15,4,-0.900969,-0.707107,15,125,54,31,12,61
278076,0.0,0.007083,0.006013,270.0,-55,0.0,0.0,2,0.0,0.0,15,4,-0.900969,-0.707107,15,125,54,36,12,61


In [29]:

y_pred = rf.predict(X)
    
    # Evaluate the model
print(classification_report(y, y_pred))
f1 = f1_score(y, y_pred, average='weighted')  # Assuming multiclass classification

print(f"F1 score: {f1:.4f}")

              precision    recall  f1-score   support

           0       0.93      0.73      0.82     69216
           1       0.53      0.99      0.69     30780
           2       0.95      0.90      0.92    235251
           3       0.44      0.98      0.61      5016
           5       0.90      0.87      0.88    162839

    accuracy                           0.87    503102
   macro avg       0.75      0.89      0.79    503102
weighted avg       0.90      0.87      0.88    503102

F1 score: 0.8782


In [21]:
X

,speed_gps,lon,lat,direction,alt,accel_angular_nn,accel_vertical_nn,location_cluster,accel_forward_nn,accel_braking_nn,day,day_of_week,day_of_week_cos,hour_cos,hour,mdm_object_name,day_of_week_sin
278480,0.0,0.005753,0.004778,326.0,-50,0.0,0.0,2,0.0,0.0,9,2,-0.222521,-0.500000,8,1038,0.974928
278481,0.0,0.005753,0.004778,326.0,-50,0.0,0.0,2,0.0,0.0,9,2,-0.222521,-0.500000,8,1038,0.974928
278482,0.0,0.005753,0.004778,326.0,-50,0.0,0.0,2,0.0,0.0,9,2,-0.222521,-0.500000,8,1038,0.974928
278483,0.0,0.005753,0.004778,326.0,-50,0.0,0.0,2,0.0,0.0,9,2,-0.222521,-0.500000,8,1038,0.974928
278484,0.0,0.005753,0.004778,326.0,-50,0.0,0.0,2,0.0,0.0,9,2,-0.222521,-0.500000,8,1038,0.974928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278073,0.0,0.007083,0.006013,270.0,-55,0.0,0.0,2,0.0,0.0,15,4,-0.900969,-0.707107,15,125,-0.433884
278074,0.0,0.007083,0.006013,270.0,-55,0.0,0.0,2,0.0,0.0,15,4,-0.900969,-0.707107,15,125,-0.433884
278075,0.0,0.007083,0.006013,270.0,-55,0.0,0.0,2,0.0,0.0,15,4,-0.900969,-0.707107,15,125,-0.433884
278076,0.0,0.007083,0.006013,270.0,-55,0.0,0.0,2,0.0,0.0,15,4,-0.900969,-0.707107,15,125,-0.433884


In [30]:
X_valid = valid[best_feature_set]
y_valid_pred = rf.predict(X_valid)

In [31]:
# Step 3: Prepare the submission DataFrame
submission_df = valid[['create_dt', 'mdm_object_name']].copy()  # Keep the required columns
submission_df['operation_kind_id'] = y_valid_pred  # Assign predictions to 'operation_kind_id'

# Save the submission DataFrame to a CSV file
submission_df.to_csv('submission.csv', index=False)

print("Submission file 'submission.csv' created successfully.")

# Step 4: Merge the submission with the original test set (telemetry_for_operations_validation.csv)
sub1 = pd.read_csv('dataset/telemetry_for_operations_validation.csv')
sub = pd.read_csv('submission.csv')

# Perform a left merge on 'create_dt' and 'mdm_object_name'
final_sub = pd.merge(sub1, sub[['create_dt', 'mdm_object_name', 'operation_kind_id']], 
                     on=['create_dt', 'mdm_object_name'], 
                     how='left')

# Fill missing 'operation_kind_id' values with 0 (as per your requirement)
final_sub['operation_kind_id'].fillna(0, inplace=True)

# Ensure 'operation_kind_id' is an integer
final_sub['operation_kind_id'] = final_sub['operation_kind_id'].astype(int)

# Save the final result as a CSV file for submission
final_sub.to_csv('final_submission.csv', index=False)

print("Final submission saved as 'final_submission.csv'")

Submission file 'submission.csv' created successfully.
Final submission saved as 'final_submission.csv'


In [8]:
import lightgbm as lgb
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import TimeSeriesSplit, train_test_split
import optuna
import matplotlib.pyplot as plt
import numpy as np
import logging
import joblib

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Prepare features and labels
X = df.drop(columns=['label', 'create_dt'])  # Features
y = df['label']

# Create a holdout test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Define a function for optimizing hyperparameters
def objective(trial):
    try:
        # Define hyperparameters to tune
        params = {
            'objective': 'multiclass',
            'num_class': len(set(y_train)),  # Number of classes
            'boosting_type': 'gbdt',
            'metric': 'multi_logloss',
            'verbosity': -1,
            'random_state': 42,
            'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
            'learning_rate': trial.suggest_float('learning_rate', 1e-5, 0.5, log=True),
            'num_leaves': trial.suggest_int('num_leaves', 20, 150),
            'max_depth': trial.suggest_int('max_depth', -1, 15),
            'min_child_samples': trial.suggest_int('min_child_samples', 1, 100),
            'feature_fraction': trial.suggest_float('feature_fraction', 0.1, 1.0),
            'bagging_fraction': trial.suggest_float('bagging_fraction', 0.1, 1.0),
            'bagging_freq': trial.suggest_int('bagging_freq', 1, 10),
            'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
            'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
        }
        
        # Initialize LightGBM model
        model = lgb.LGBMClassifier(**params)

        # Time-based split for cross-validation
        tscv = TimeSeriesSplit(n_splits=5)

        # Store the evaluation results
        f1_scores = []

        # Cross-validation loop
        for train_index, val_index in tscv.split(X_train):
            X_train_split, X_val = X_train.iloc[train_index], X_train.iloc[val_index]
            y_train_split, y_val = y_train.iloc[train_index], y_train.iloc[val_index]

            # Fit the model with early stopping
            evals = [(X_train_split, y_train_split), (X_val, y_val)]  # Validation data
            model.fit(X_train_split, y_train_split, 
                      eval_set=evals, 
                      eval_metric='multi_logloss',
                      callbacks=[lgb.early_stopping(stopping_rounds=100)])

            # Make predictions on the validation data
            y_pred = model.predict(X_val)

            # Evaluate the model using F1 score and store it
            f1 = f1_score(y_val, y_pred, average='weighted')
            f1_scores.append(f1)

        # Return the mean F1 score across all splits
        return np.mean(f1_scores)

    except Exception as e:
        logger.error(f"Error in objective function: {str(e)}")
        return None

# Run the Optuna optimization
study = optuna.create_study(direction='maximize')
try:
    study.optimize(objective, n_trials=50, n_jobs=-1)  # Reduced number of trials
except Exception as e:
    logger.error(f"Error during optimization: {str(e)}")

# Print the best parameters and score
logger.info(f"Best parameters: {study.best_params}")
logger.info(f"Best score: {study.best_value}")

# Retrain the model using the best parameters
best_params = study.best_params
final_model = lgb.LGBMClassifier(**best_params)

# Fit the final model on the entire training dataset
evals = [(X_train, y_train), (X_test, y_test)]  # Include test set for final evaluation
final_model.fit(X_train, y_train, eval_set=evals, eval_metric='multi_logloss')

# Evaluate on the test set
y_pred = final_model.predict(X_test)
test_f1 = f1_score(y_test, y_pred, average='weighted')
logger.info(f"Final model F1 score on test set: {test_f1}")

# Feature importance analysis
feature_importance = final_model.feature_importances_
feature_names = X_train.columns
feature_importance_dict = dict(zip(feature_names, feature_importance))
sorted_features = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)
logger.info("Top 10 most important features:")
for feature, importance in sorted_features[:10]:
    logger.info(f"{feature}: {importance}")

# Plot feature importance
plt.figure(figsize=(10, 6))
plt.bar(range(len(feature_importance)), feature_importance)
plt.title('Feature Importance')
plt.xlabel('Features')
plt.ylabel('Importance')
plt.xticks(range(len(feature_importance)), feature_names, rotation=90)
plt.tight_layout()
plt.savefig('feature_importance.png')
plt.close()

# Save the model using joblib
try:
    joblib.dump(final_model, 'lightgbm_model.pkl')
    logger.info("Model saved successfully.")
except Exception as e:
    logger.error(f"Error saving model: {str(e)}")

# Plot the F1 scores across the trials
plt.figure(figsize=(10, 6))
plt.plot(study.trials_dataframe()['number'], study.trials_dataframe()['value'], marker='o')
plt.title('F1 Score across Optuna Trials')
plt.xlabel('Trial Number')
plt.ylabel('F1 Score')
plt.grid()
plt.savefig('optimization_progress.png')
plt.close()

logger.info("Optimization and training process completed.")

[I 2024-09-30 02:30:27,384] A new study created in memory with name: no-name-d2180410-e5fd-4c9f-902a-8f86ad9aeb79


Training until validation scores don't improve for 100 rounds
Training until validation scores don't improve for 100 rounds
Training until validation scores don't improve for 100 rounds
Training until validation scores don't improve for 100 rounds
Training until validation scores don't improve for 100 rounds
Training until validation scores don't improve for 100 rounds
Training until validation scores don't improve for 100 rounds
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[3]	training's multi_logloss: 0.366902	valid_1's multi_logloss: 1.05335
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[215]	training's multi_logloss: 0.372736	valid_1's multi_logloss: 0.739946
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[7]	training's multi_logloss: 0.277632	valid_1's multi_logloss: 0.577434
Training until validation scores don't improve for 100 

[I 2024-09-30 02:32:28,746] Trial 5 finished with value: 0.7376786856777423 and parameters: {'n_estimators': 223, 'learning_rate': 0.45600545369529105, 'num_leaves': 25, 'max_depth': 8, 'min_child_samples': 98, 'feature_fraction': 0.5200252404743344, 'bagging_fraction': 0.7023278201635385, 'bagging_freq': 10, 'lambda_l1': 0.0028104535960624955, 'lambda_l2': 6.336132038343484}. Best is trial 5 with value: 0.7376786856777423.


Early stopping, best iteration is:
[2]	training's multi_logloss: 0.706809	valid_1's multi_logloss: 0.812022
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[188]	training's multi_logloss: 1.15244	valid_1's multi_logloss: 1.38749
Training until validation scores don't improve for 100 rounds
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[73]	training's multi_logloss: 0.210203	valid_1's multi_logloss: 0.914436
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[465]	training's multi_logloss: 0.356011	valid_1's multi_logloss: 0.518454
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[188]	training's multi_logloss: 1.23528	valid_1's multi_logloss: 1.2034
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:

[I 2024-09-30 02:35:53,220] Trial 3 finished with value: 0.7692083531244016 and parameters: {'n_estimators': 465, 'learning_rate': 0.010519359047725548, 'num_leaves': 50, 'max_depth': 4, 'min_child_samples': 62, 'feature_fraction': 0.9851258370643747, 'bagging_fraction': 0.49919040697623374, 'bagging_freq': 10, 'lambda_l1': 0.000639672386683225, 'lambda_l2': 0.0003126231267786106}. Best is trial 3 with value: 0.7692083531244016.


...

[I 2024-09-30 03:40:33,844] Trial 46 finished with value: 0.7667172374391324 and parameters: {'n_estimators': 990, 'learning_rate': 0.017388132306042192, 'num_leaves': 107, 'max_depth': 13, 'min_child_samples': 57, 'feature_fraction': 0.5781463137714453, 'bagging_fraction': 0.2282264351373045, 'bagging_freq': 9, 'lambda_l1': 0.026198336773175, 'lambda_l2': 2.627716118425245e-05}. Best is trial 34 with value: 0.7900832664632056.


Did not meet early stopping. Best iteration is:
[478]	training's multi_logloss: 0.180125	valid_1's multi_logloss: 0.446322
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[445]	training's multi_logloss: 0.184858	valid_1's multi_logloss: 0.520277


[I 2024-09-30 03:41:57,795] Trial 48 finished with value: 0.7857104737820091 and parameters: {'n_estimators': 543, 'learning_rate': 0.009469484614236131, 'num_leaves': 116, 'max_depth': 10, 'min_child_samples': 54, 'feature_fraction': 0.6355493514287041, 'bagging_fraction': 0.3474230745409397, 'bagging_freq': 6, 'lambda_l1': 7.665429339382784, 'lambda_l2': 8.391154158656135e-08}. Best is trial 34 with value: 0.7900832664632056.
2024-09-30 03:41:57,797 - INFO - Best parameters: {'n_estimators': 639, 'learning_rate': 0.02650299032439699, 'num_leaves': 100, 'max_depth': 9, 'min_child_samples': 72, 'feature_fraction': 0.6374815078623395, 'bagging_fraction': 0.14893661175532902, 'bagging_freq': 7, 'lambda_l1': 6.412312015009089, 'lambda_l2': 1.5255322598961795e-07}
2024-09-30 03:41:57,798 - INFO - Best score: 0.7900832664632056


[LightGBM] [Warning] feature_fraction is set=0.6374815078623395, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6374815078623395
[LightGBM] [Warning] lambda_l1 is set=6.412312015009089, reg_alpha=0.0 will be ignored. Current value: lambda_l1=6.412312015009089
[LightGBM] [Warning] lambda_l2 is set=1.5255322598961795e-07, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.5255322598961795e-07
[LightGBM] [Warning] bagging_fraction is set=0.14893661175532902, subsample=1.0 will be ignored. Current value: bagging_fraction=0.14893661175532902
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] feature_fraction is set=0.6374815078623395, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6374815078623395
[LightGBM] [Warning] lambda_l1 is set=6.412312015009089, reg_alpha=0.0 will be ignored. Current value: lambda_l1=6.412312015009089
[LightGBM] [Warning] lambda_l2 

2024-09-30 03:43:36,601 - INFO - Final model F1 score on test set: 0.7377020031716888
2024-09-30 03:43:36,605 - INFO - Top 10 most important features:
2024-09-30 03:43:36,605 - INFO - lon: 28606
2024-09-30 03:43:36,606 - INFO - lat: 27813
2024-09-30 03:43:36,606 - INFO - alt: 24729
2024-09-30 03:43:36,607 - INFO - direction: 23657
2024-09-30 03:43:36,608 - INFO - minute: 22883
2024-09-30 03:43:36,608 - INFO - hour: 11527
2024-09-30 03:43:36,609 - INFO - second: 9856
2024-09-30 03:43:36,610 - INFO - hour_sin: 9786
2024-09-30 03:43:36,610 - INFO - hour_cos: 9344
2024-09-30 03:43:36,611 - INFO - speed_gps: 9257
2024-09-30 03:43:37,569 - INFO - Model saved successfully.
2024-09-30 03:43:37,727 - INFO - Optimization and training process completed.


In [9]:

y_pred = final_model.predict(X)
    
    # Evaluate the model
print(classification_report(y, y_pred))
f1 = f1_score(y, y_pred, average='weighted')  # Assuming multiclass classification

print(f"F1 score: {f1:.4f}")

[LightGBM] [Warning] feature_fraction is set=0.6374815078623395, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6374815078623395
[LightGBM] [Warning] lambda_l1 is set=6.412312015009089, reg_alpha=0.0 will be ignored. Current value: lambda_l1=6.412312015009089
[LightGBM] [Warning] lambda_l2 is set=1.5255322598961795e-07, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.5255322598961795e-07
[LightGBM] [Warning] bagging_fraction is set=0.14893661175532902, subsample=1.0 will be ignored. Current value: bagging_fraction=0.14893661175532902
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
              precision    recall  f1-score   support

           0       0.89      0.88      0.88     69216
           1       0.82      0.85      0.83     30780
           2       0.94      0.94      0.94    235251
           3       0.66      0.76      0.71      5016
           5       0.93      0.92      0.93  

In [11]:
X_valid = valid.drop(columns=[ 'create_dt'])
y_valid_pred = final_model.predict(X_valid)

[LightGBM] [Warning] feature_fraction is set=0.6374815078623395, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6374815078623395
[LightGBM] [Warning] lambda_l1 is set=6.412312015009089, reg_alpha=0.0 will be ignored. Current value: lambda_l1=6.412312015009089
[LightGBM] [Warning] lambda_l2 is set=1.5255322598961795e-07, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.5255322598961795e-07
[LightGBM] [Warning] bagging_fraction is set=0.14893661175532902, subsample=1.0 will be ignored. Current value: bagging_fraction=0.14893661175532902
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7


In [12]:
# Step 3: Prepare the submission DataFrame
submission_df = valid[['create_dt', 'mdm_object_name']].copy()  # Keep the required columns
submission_df['operation_kind_id'] = y_valid_pred  # Assign predictions to 'operation_kind_id'

# Save the submission DataFrame to a CSV file
submission_df.to_csv('submission.csv', index=False)

print("Submission file 'submission.csv' created successfully.")

# Step 4: Merge the submission with the original test set (telemetry_for_operations_validation.csv)
sub1 = pd.read_csv('dataset/telemetry_for_operations_validation.csv')
sub = pd.read_csv('submission.csv')

# Perform a left merge on 'create_dt' and 'mdm_object_name'
final_sub = pd.merge(sub1, sub[['create_dt', 'mdm_object_name', 'operation_kind_id']], 
                     on=['create_dt', 'mdm_object_name'], 
                     how='left')

# Fill missing 'operation_kind_id' values with 0 (as per your requirement)
final_sub['operation_kind_id'].fillna(0, inplace=True)

# Ensure 'operation_kind_id' is an integer
final_sub['operation_kind_id'] = final_sub['operation_kind_id'].astype(int)

# Save the final result as a CSV file for submission
final_sub.to_csv('final_submission.csv', index=False)

print("Final submission saved as 'final_submission.csv'")

Submission file 'submission.csv' created successfully.
Final submission saved as 'final_submission.csv'


In [13]:
print(best_params)

{'n_estimators': 639, 'learning_rate': 0.02650299032439699, 'num_leaves': 100, 'max_depth': 9, 'min_child_samples': 72, 'feature_fraction': 0.6374815078623395, 'bagging_fraction': 0.14893661175532902, 'bagging_freq': 7, 'lambda_l1': 6.412312015009089, 'lambda_l2': 1.5255322598961795e-07}
